## 1: Introduction
* The problem 
    * Quality > Performance
* Existing solutions
    * cropping as a service
        https://github.com/thumbor/thumbor
    
* What I want
    * self-contained
    * flexible
    * tweakable

## 2: Tools – OpenCV

* What is it?
* Specific libraries:
    * Haar-cascade classifiers
    http://docs.opencv.org/3.1.0/d7/d8b/tutorial_py_face_detection.html#gsc.tab=0
    https://en.wikipedia.org/wiki/Viola%E2%80%93Jones_object_detection_framework
    * ORB feature detection
    https://en.wikipedia.org/wiki/ORB_(feature_descriptor)
    http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_orb/py_orb.html
* How to install
    http://www.pyimagesearch.com/2015/07/20/install-opencv-3-0-and-python-3-4-on-ubuntu/
* Read more
    pyimagesearch
    opencv docs

## 3: Dummy feature detector
* What output do I want?

* code 
* json output
* react widget
* discussion:
    * How can I use this?

## 4: Feature detector
* What is ORB?
* link to documentation
* code
* output
* discussion

## 5: Face detection
* What are Haas-cascades?
* code
* output
* discussion

## 6: Combined feature detector
* explanation
* code 
* output
* discussion

## 7: Summary
* it's easy!
* links
    * OpenCV guy
    * documentation
    * sorl-thumbnail
* My own stuff
    * Algorithm
    * React component

In [9]:
from IPython.display import HTML
from utils import cropengine, reactjs
import json
HTML(reactjs.css())

In [6]:
import cv2
from utils.boundingbox import Box
from typing import List, Dict, Union
from collections import OrderedDict
import numpy

FileName = str  # for type annotation


def opencv_image(image_file: FileName, maxsize: int) -> numpy.ndarray:
    """Read a image file, convert to grayscale opencv image format

    Our OpenCV algorithms works on a two dimensional numpy array integers
    where 0 is black and 255 is white. We don't care about the colour channels.
    """
    cv_image = cv2.imread(image_file)
    imgheight, imgwidth = cv_image.shape[:2]
    if imgwidth > imgheight:
        dimensions = maxsize, maxsize * imgheight // imgwidth
    else:
        dimensions = maxsize * imgwidth // imgheight, maxsize
    cv_image = cv2.resize(cv_image, dimensions)
    return cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)


def normalize_box(box: Box, image_width: int, image_height: int) -> Box:
    """ Converts a Box to a relative coordinate system from 0,0 to 1,1

    The input box, width and height are the actual pixel values of the input
    image. The output will be in a normalized coordinate system where the image
    width and height are both 1. Any part of the Box that overflows the image
    frame is truncated. """
    return Box(0, 0, 1, 1) & Box(
        left=box.left / image_width,
        top=box.top / image_height,
        right=box.right / image_width,
        bottom=box.bottom / image_height,
    )


class Feature(Box):
    """Salient feature in an image"""

    def __init__(self, value: float, className: str, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.value = value
        self.className = className

    def serialize(self) -> Dict[str, Union[str, float]]:
        """Create a dictionary with the properties needed to visualize the
        feature"""

        return OrderedDict([
            ('className', self.className),
            ('x', round(self.left, 3)),
            ('y', round(self.top, 3)),
            ('width', round(self.width, 3)),
            ('height', round(self.height, 3)),
            ('value', round(self.value, 3)),
        ])


class BaseCropEngine:

    def __init__(self, padding: float=0.6) -> None:
        self.padding = padding

    def find_features(self, image_file: FileName) -> List[Feature]:
        """ Finds the most salient features of the image """
        box = Box(0, 0, 1, 1) * self.padding
        middle = Feature(1, 'base', **box.geometry)
        return [middle]


class KeypointCropEngine(BaseCropEngine):

    def __init__(
        self, n: int=10, imgsize: int=300, padding: float=1.0, **kwargs
    ) -> None:
        self.imagesize = imgsize
        self.padding = padding
        self.number = n
        arguments = dict(
            nfeatures=self.number + 1,
            scaleFactor=1.4,
            patchSize=self.imagesize // 8,
            edgeThreshold=self.imagesize // 8,
            WTA_K=2,
            scoreType=cv2.ORB_FAST_SCORE,
        )
        arguments.update(kwargs)
        self.feature_detector = cv2.ORB_create(**arguments)

    def find_features(self, image_file: FileName) -> List[Feature]:
        features = []
        cv_image = opencv_image(image_file, self.imagesize)
        imgheight, imgwidth = cv_image.shape[:2]
        keypoints = self.feature_detector.detectAndCompute(
            image=cv_image, mask=None)[0]

        def normalize_value(box: Box, kp: cv2.KeyPoint) -> float:
            return 0.001 * box.size * kp.response ** 2

        for keypoint in keypoints:
            r = keypoint.size / 2
            x, y = keypoint.pt
            box = Box(x - r, y - r, x + r, y + r) * self.padding
            box = normalize_box(box, imgwidth, imgheight)
            feature = Feature(
                value=normalize_value(box, keypoint),
                className='ORB keypoint',
                **box.geometry,
            )
            features.append(feature)

        features.sort(key=lambda b: -b.value)
        return features[:self.number]
    
#     def plot_keypoints(self, image_file: FileName) -> None:
#         cv_image = opencv_image(image_file, self.imagesize)
#         keypoints = self.feature_detector.detectAndCompute(
#             image=cv_image, mask=None)[0]
#         cv_image = cv2.drawKeypoints(
#             image=cv_image, keypoints=keypoints, outImage=cv_image, 
#             flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
#         return cv_image

    

# import uuid, os
# img = KeypointCropEngine().plot_keypoints('./kids_in_the_hall.jpg')
# fn = 'tmp-{}.jpg'.format(uuid.uuid1().hex)
# cv2.imwrite(fn, img)
# HTML('<img src={} />'.format(fn))

In [7]:
def croppify(image_file, crop_engine = KeypointCropEngine()):
    features = crop_engine.find_features(image_file)
    # print (json.dumps([f.serialize() for f in features], indent=2))
    return HTML(reactjs.render(image_file, features))


In [8]:
#os.remove(fn)
croppify('./kids_in_the_hall.jpg', KeypointCropEngine(n=30))

In [10]:
croppify('./kids_in_the_hall.jpg', cropengine.FaceCropEngine())

In [11]:
croppify('11-NYH-nyhklarefors-01-SGS.jpg', KeypointCropEngine(imgsize=200, n=20))

In [ ]:
croppify('./kids_in_the_hall.jpg', cropengine.KeypointCropEngine())

In [ ]:
croppify('./forlag.jpg', cropengine.FaceCropEngine())